In [1]:
pip install yfinance

  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=6dedb904811dd1c6e8a95110ae1b1e899cd13b43f72fb0f9027e19e0afb7d4b1
  Stored in directory: /root/.cache/pip/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
Successfully built yfinance


In [2]:
import yfinance as yf
import numpy as np

In [3]:
class dynamic_NN:

  def __init__(self, X, Y, **kwargs):
    
    #usr should explicitly define layer0 to last layer
    self.__NN_layer_dict = kwargs
    self.__Y = Y
    #theta is dict with 'layer0': 5*3 ...
    self.__theta = self.construct_theta()
    self.__forward_prop = {'layer0':X}
    self.__m = len(Y)

  def construct_theta(self):

    theta = {}
    epsilon_init = 0.12
    for layer in range(len(self.__NN_layer_dict)-1):
      theta['layer'+str(layer)] = np.random.rand(self.__NN_layer_dict['layer'+str(layer+1)], self.__NN_layer_dict['layer'+str(layer)]+1)*epsilon_init -epsilon_init

    return theta

  def activate(self,method,X):
    #allow user to choose activation method in each layer from layer 1
    if method == 'sigmoid':
      result = 1/(1+np.exp(-1*X))

    return result

  ##add one to the layer
  def add_ones(self,layer_X):

    _,num_sample = np.shape(layer_X)
    ones = np.ones((1,num_sample))
    result_X = np.insert(layer_X,0,ones,axis=0)

    return result_X

  def forward_propagation(self,methods):

    #for i = 1 to len(NN_dict)
    for layer_num in range(1,len(self.__NN_layer_dict)):

      #compute i layer using forward prop i-1
      preparation = self.add_ones(self.__forward_prop['layer'+str(layer_num-1)])
      Z_val = np.dot(self.__theta['layer'+str(layer_num-1)], preparation)

      #allow user to choose activation method in each layer from layer 1
      Activation = self.activate(methods['layer'+str(layer_num)],Z_val)

      self.__forward_prop['layer'+str(layer_num)] = Activation
  
  def compute_cost(self, penalty_lambda):
    
    #compute last layer hypothesis
    pos = len(self.__NN_layer_dict) - 1 #find the label of last layer
    hypos = self.__forward_prop['layer'+str(pos)]

    #compute logged hypothesis
    log_hypos = np.log(hypos)

    #compute sum theta sq
    sum_theta_sq = 0
    for layer in self.__NN_layer_dict:

      temp = self.__NN_layer_dict[layer]
      temp = np.square(temp)
      sum_theta_sq = sum_theta_sq + np.sum(temp)

    #compute regularization term
    regularization = penalty_lambda/(2*self.__m)*sum_theta_sq
    cost = -1/self.__m * (np.dot(self.__Y, log_hypos.transpose()) + np.dot(1-self.__Y, (1-log_hypos).transpose())) + regularization
    
    return cost

  def sigmoid_gradient(self,Z):
    
    gradient = np.multiply(Z,1-Z)
    return gradient

  def backward_propagation(self,penalty_lambda):

    pos = len(self.__NN_layer_dict) - 1 #find the label of last layer (if num-of-layer = 3, pos = 2) 
    hypos = self.__forward_prop['layer'+str(pos)] 

    delta_last_layer = self.__Y - hypos
    delta = {'layer'+str(pos):delta_last_layer}

    #from layer last-1 to 1 find delta(i.e from 1 to 1)
    for layer in reversed(range(1,pos)):
      delta_temp = np.dot(self.__theta['layer'+str(layer)][:,1:].transpose(), delta['layer'+str(layer+1)])
    ##if method is not sigmoid, make an if statement to select right method
      delta_temp = np.multiply(delta_temp, self.sigmoid_gradient(self.__forward_prop['layer'+str(layer)]))
      delta['layer'+str(layer)] = delta_temp

    gradient = {}
    #i.e from 0 to 1
    for layer in range(pos):
      activation = self.add_ones(self.__forward_prop['layer'+str(layer)])
      gradient_temp = 1/self.__m * np.dot(delta['layer'+str(layer+1)], activation.transpose())

      theta_no_bias = self.__theta['layer'+str(layer)][:,1:]
      gradient_temp[:,1:] = gradient_temp[:,1:] + penalty_lambda/self.__m* theta_no_bias

      gradient['layer'+str(layer)] = gradient_temp

    return gradient

  def start_learning(self,**kwargs):

    self.forward_propagation(kwargs)

    return self.__forward_prop
    

  def get_NN_layer_dict(self):

    print(self.__NN_layer_dict)

  def get_theta(self):

    print(self.__theta)
  

In [4]:
def handle_data(Y,*args):

    def __set_Y(Y):
    
      Y[Y>0] = 1
      Y[Y<0] = 0
      return Y

    Y = Y.to_numpy()
    Y_reshaped = Y.reshape((1,len(Y)))
    Y_reshaped = (Y_reshaped-Y_reshaped.mean())/np.std(Y_reshaped)
    final_Y = __set_Y(Y_reshaped)
    ##get empty x array
    X = np.array([])
    for arg in args:

      arg = arg.to_numpy()
      arg_reshaped = arg.reshape((1,len(arg)))
      arg_reshaped = (arg_reshaped-arg_reshaped.mean())/np.std(arg_reshaped)
      X = np.append(X,[arg_reshaped])

    #reshape the appended x to the shape we want
    X_reshaped = X.reshape((len(args),len(Y)))

    return X_reshaped,Y_reshaped

In [6]:
start = '2012-12-10'
end = '2012-12-30'
Y = yf.download('AAPL',start=start,end=end,interval='1d')['Adj Close']
X = yf.download('^GSPC',start=start,end=end,interval='1d')
X_price = X['Adj Close']
X_vol = X['Volume']
X_open = X['Open']

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [7]:
x,y = handle_data(Y,X_open,X_price,X_vol)

In [8]:
test_case = dynamic_NN(x,y,layer0 = 3,layer1=4,layer2=4,layer3=1)

In [9]:
test_case.start_learning(layer1 = 'sigmoid',layer2 = 'sigmoid', layer3 = 'sigmoid')

{'layer0': array([[-0.9251776 , -0.87520186,  0.09182188,  0.15844351, -0.78152751,
         -1.39671334,  0.36558936,  2.06440901,  0.92145997,  1.73963654,
          0.3322849 , -0.03100181, -0.74197132, -0.92205174],
        [-0.64339179,  0.17683339,  0.23334156, -0.56393763, -1.08221113,
          0.39933091,  1.84997292,  0.88052868,  1.57625915,  0.38079304,
          0.07265489, -0.53038624, -0.68312965, -2.0666581 ],
        [-0.33102449,  0.37678773,  0.44076055,  0.05021309, -0.10182297,
          0.16511838,  1.08591594,  0.61559278,  0.4163221 ,  2.09403717,
         -2.23484139, -1.10800827, -0.51510134, -0.95394926]]),
 'layer1': array([[0.53663839, 0.49766077, 0.47845925, 0.50576524, 0.53688426,
         0.5078168 , 0.42118689, 0.4271351 , 0.43702409, 0.40073552,
         0.5559602 , 0.54202287, 0.53979915, 0.58441598],
        [0.49246286, 0.47255625, 0.46139975, 0.47564658, 0.49341333,
         0.47780416, 0.43061526, 0.43175468, 0.43713894, 0.42189217,
         0.496

In [10]:
test_case.compute_cost(1)

array([[11.42941578]])

In [11]:
test_case.backward_propagation(1)

{'layer0': array([[-0.00059152, -0.06526627, -0.08208947, -0.11335439],
        [-0.00048459, -0.03886258, -0.0468434 , -0.04985702],
        [-0.00067953, -0.10241573, -0.02031364, -0.03698147],
        [-0.00057959, -0.06466375, -0.05426893, -0.09339349]]),
 'layer1': array([[ 0.0105928 , -0.05842312, -0.05953742, -0.10025483, -0.00998055],
        [ 0.00826097, -0.09740671, -0.07312332, -0.00370429, -0.10348392],
        [ 0.0090555 ,  0.0031882 , -0.01819317, -0.07233361, -0.05219031],
        [ 0.00832594, -0.10291153, -0.03515285, -0.09043696, -0.07557158]]),
 'layer2': array([[-0.3763822 , -0.27781799, -0.25368912, -0.2672951 , -0.25388121]])}